En este código se utiliza el modelo de lenguaje BERT para generar vectores, que posteriormente nos ayudarán a calcular distancias.

# Utilizaremos BERT para generar embeddings y calcular distancias

Cargamos el df y el modelo

In [1]:
import pandas as pd

In [2]:
from google.colab import drive

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_csv('/content/drive/MyDrive/peru_larco/Data/base_clean/base_global.csv',  sep=';')
df

,Unnamed: 0,catalogation_id,cronology,cronology_time,culture_cl,morfofunctional_category,description,principal_scene,decoration_tecnique_external_body_section1,color_external_body_section1,...,trait_n101,trait_n102,trait_n103,trait_n104,trait_n105,file_path,image_path,description_length,tokens,numero_palabras
0,0,ML020107,Horizonte Medio,7,Sican,botella doble cuerpo asa puente cintada silbadora,botella doble cuerpo asa puente cintada silbad...,NaN,pintado escultorico,crema y naranja,...,1,1,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020107a.jpg,191,botella doble cuerpo asa puente cintada silbad...,21
1,1,ML020108,Horizonte Medio,7,Sican,botella doble pico asa puente cintada escultorica,botella doble pico asa puente cintada escultor...,NaN,pintado escultorico,rojo y naranja,...,1,1,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020108a.jpg,245,botella doble pico asa puente cintada escultor...,29
2,2,ML020109,Horizonte Medio,7,Sican,botella gollete asa puente cintada protoma sil...,botella gollete asa puente cintada protoma sil...,NaN,pintado escultorico,crema y naranja,...,1,1,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020109a.jpg,134,botella gollete asa puente cintada protoma sil...,16
3,3,ML020110,Horizonte Medio,7,Sican,botella gollete asa puente cintada protoma sil...,botella gollete asa puente cintada protoma sil...,NaN,pintado escultorico,crema y naranja,...,1,1,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020110a.jpg,125,botella gollete asa puente cintada protoma sil...,15
4,4,ML020111,Horizonte Medio,7,Sican,botella gollete asa puente cintada protoma sil...,botella gollete asa puente cintada protoma sil...,NaN,pintado escultorico,crema y marron,...,1,1,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020111a.jpg,158,botella gollete asa puente cintada protoma sil...,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32623,33576,ML035498,Período Intermedio Temprano,6,Tiahuanaco,botella gollete asa lateral,botella gollete asa lateral con disenos geomet...,NaN,pintado,"crema, negro, rojo y naranja",...,1,1,0,1,0,data/tiahuanaco_6/1 ADMINISTRADOR COLECCIONES ...,data/images/ML035498a.jpg,94,botella gollete asa lateral disenos geometrico...,10
32624,33577,ML038832,Período Intermedio Temprano,6,Tiahuanaco,plato,plato con disenos geometricos de lineas horizo...,NaN,pintado,"crema, rojo y naranja",...,1,1,0,1,0,data/tiahuanaco_6/1 ADMINISTRADOR COLECCIONES ...,data/images/ML038832a.jpg,53,plato disenos geometricos lineas horizontales,5
32625,33578,ML038833,Período Intermedio Temprano,6,Tiahuanaco,plato,plato con disenos geometricos de eses ( s) y l...,NaN,pintado,"crema, negro, rojo y naranja",...,1,1,0,1,0,data/tiahuanaco_6/1 ADMINISTRADOR COLECCIONES ...,data/images/ML038833a.jpg,62,plato disenos geometricos es s linea horizontal,7
32626,33580,ML015241,Horizonte Temprano,5,Cajamarca,cuenco,cuenco con representar de cabeza estilizada co...,NaN,pintado inciso,marron,...,1,1,0,1,0,data/cajamarca_5/1 ADMINISTRADOR COLECCIONES V...,data/images/ML015241a.jpg,63,cuenco representar cabeza estilizada ojo excen...,6


In [5]:
df = df[~df.description.isnull()]
df['description_length'] = df.description.apply(lambda x: len(x))
max(df.description_length)

1342

In [6]:
descriptions = df.tokens.tolist()

In [7]:
import numpy as np
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
model_name = 'dccuchile/bert-base-spanish-wwm-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/310 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/486k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def prepare_data_for_bert(texts, tokenizer, max_length):
    input_ids = []
    attention_masks = []
    encodeds_data = []

    for text in texts:
        encoded_data = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_length,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        input_ids.append(encoded_data['input_ids'])
        attention_masks.append(encoded_data['attention_mask'])
        encodeds_data.append(encoded_data)

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)

    return input_ids, attention_masks, encodeds_data

In [ ]:
input_ids, attention_masks, encodeds_data = prepare_data_for_bert(descriptions, tokenizer, s)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2645: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
tokenizer.convert_ids_to_tokens(encodeds_data[0]['input_ids'][0])

['[CLS]',
 'botella',
 'doble',
 'cuerpo',
 'asa',
 'puente',
 'cinta',
 '##da',
 'sil',
 '##bado',
 '##ra',
 'representar',
 'personaje',
 'con',
 'tocado',
 ',',
 'pintura',
 'facial',
 'y',
 'tapar',
 '##ra',
 '##bo',
 'que',
 'coge',
 'por',
 'los',
 'hombros',
 'a',
 'personaje',
 'con',
 'casco',
 ',',
 'pintura',
 'facial',
 ',',
 'collar',
 'y',
 'tun',
 '##ica',
 '.',
 '[SEP]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',


In [ ]:
attention_masks

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])

In [ ]:
df

,Unnamed: 0,catalogation_id,cronology,cronology_time,culture_cl,morfofunctional_category,description,principal_scene,decoration_tecnique_external_body_section1,color_external_body_section1,...,trait_n101,trait_n102,trait_n103,trait_n104,trait_n105,file_path,image_path,description_length,tokens,numero_palabras
0,0,ML020107,Horizonte Medio,7,Sican,botella doble cuerpo asa puente cintada silbadora,botella doble cuerpo asa puente cintada silbad...,NaN,pintado escultorico,crema y naranja,...,1,1,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020107a.jpg,191,botella doble cuerpo asa puente cintada silbad...,21
1,1,ML020108,Horizonte Medio,7,Sican,botella doble pico asa puente cintada escultorica,botella doble pico asa puente cintada escultor...,NaN,pintado escultorico,rojo y naranja,...,1,1,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020108a.jpg,245,botella doble pico asa puente cintada escultor...,29
2,2,ML020109,Horizonte Medio,7,Sican,botella gollete asa puente cintada protoma sil...,botella gollete asa puente cintada protoma sil...,NaN,pintado escultorico,crema y naranja,...,1,1,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020109a.jpg,134,botella gollete asa puente cintada protoma sil...,16
3,3,ML020110,Horizonte Medio,7,Sican,botella gollete asa puente cintada protoma sil...,botella gollete asa puente cintada protoma sil...,NaN,pintado escultorico,crema y naranja,...,1,1,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020110a.jpg,125,botella gollete asa puente cintada protoma sil...,15
4,4,ML020111,Horizonte Medio,7,Sican,botella gollete asa puente cintada protoma sil...,botella gollete asa puente cintada protoma sil...,NaN,pintado escultorico,crema y marron,...,1,1,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020111a.jpg,158,botella gollete asa puente cintada protoma sil...,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32623,33576,ML035498,Período Intermedio Temprano,6,Tiahuanaco,botella gollete asa lateral,botella gollete asa lateral con disenos geomet...,NaN,pintado,"crema, negro, rojo y naranja",...,1,1,0,1,0,data/tiahuanaco_6/1 ADMINISTRADOR COLECCIONES ...,data/images/ML035498a.jpg,94,botella gollete asa lateral disenos geometrico...,10
32624,33577,ML038832,Período Intermedio Temprano,6,Tiahuanaco,plato,plato con disenos geometricos de lineas horizo...,NaN,pintado,"crema, rojo y naranja",...,1,1,0,1,0,data/tiahuanaco_6/1 ADMINISTRADOR COLECCIONES ...,data/images/ML038832a.jpg,53,plato disenos geometricos lineas horizontales,5
32625,33578,ML038833,Período Intermedio Temprano,6,Tiahuanaco,plato,plato con disenos geometricos de eses ( s) y l...,NaN,pintado,"crema, negro, rojo y naranja",...,1,1,0,1,0,data/tiahuanaco_6/1 ADMINISTRADOR COLECCIONES ...,data/images/ML038833a.jpg,62,plato disenos geometricos es s linea horizontal,7
32626,33580,ML015241,Horizonte Temprano,5,Cajamarca,cuenco,cuenco con representar de cabeza estilizada co...,NaN,pintado inciso,marron,...,1,1,0,1,0,data/cajamarca_5/1 ADMINISTRADOR COLECCIONES V...,data/images/ML015241a.jpg,63,cuenco representar cabeza estilizada ojo excen...,6


In [ ]:
df['tokens'] = [' '.join(tokenizer.convert_ids_to_tokens(encodeds_data[i]['input_ids'][0])) for i in range(len(encodeds_data))]

In [ ]:
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
dataset = TensorDataset(input_ids, attention_masks)
data_loader = DataLoader(dataset, batch_size=8, shuffle=True)

# Decidir donde se cargará el modelo, CPU o GPU

In [ ]:
torch.cuda.is_available()

True

In [ ]:
torch.cuda.memory_allocated()

439936000

In [ ]:
torch.cuda.empty_cache()

In [ ]:
device = torch.device('cuda')

In [ ]:
model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(31002, 768, padding_idx=1)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [ ]:
def get_embeddings(model, data_loader):
    model.eval()
    embeds_CLSs = []
    embeds_mean = []

    for i, batch in enumerate(data_loader):
        print(f'Batch: {i}')

        b_input_ids = batch[0].to(device)
        b_attention_masks = batch[1].to(device)

        with torch.no_grad():
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_attention_masks)[0]
            CLSs = outputs[:, 0, :]
            means = outputs.mean(dim=1)

            # Guardamos los embeddings
            embeds_CLSs.append(CLSs)
            embeds_mean.append(means)

# Cargamos los batches

In [ ]:
model.eval()
embeds_CLSs = []
embeds_mean = []
embeds_max_over_time = []

for i, batch in enumerate(data_loader):
    print(f'Batch: {i}')

    b_input_ids = batch[0].to(device)
    b_attention_masks = batch[1].to(device)

    with torch.no_grad():
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_attention_masks)[0]
        CLSs = outputs[:, 0, :]
        means = outputs.mean(dim=1)
        max_over_time = outputs.max(dim=1)

        # Guardamos los embeddings
        embeds_CLSs.append(CLSs)
        embeds_mean.append(means)
        embeds_max_over_time.append(max_over_time)

Batch: 0
Batch: 1
Batch: 2
Batch: 3
Batch: 4
Batch: 5
Batch: 6
Batch: 7
Batch: 8
Batch: 9
Batch: 10
Batch: 11
Batch: 12
Batch: 13
Batch: 14
Batch: 15
Batch: 16
Batch: 17
Batch: 18
Batch: 19
Batch: 20
Batch: 21
Batch: 22
Batch: 23
Batch: 24
Batch: 25
Batch: 26
Batch: 27
Batch: 28
Batch: 29
Batch: 30
Batch: 31
Batch: 32
Batch: 33
Batch: 34
Batch: 35
Batch: 36
Batch: 37
Batch: 38
Batch: 39
Batch: 40
Batch: 41
Batch: 42
Batch: 43
Batch: 44
Batch: 45
Batch: 46
Batch: 47
Batch: 48
Batch: 49
Batch: 50
Batch: 51
Batch: 52
Batch: 53
Batch: 54
Batch: 55
Batch: 56
Batch: 57
Batch: 58
Batch: 59
Batch: 60
Batch: 61
Batch: 62
Batch: 63
Batch: 64
Batch: 65
Batch: 66
Batch: 67
Batch: 68
Batch: 69
Batch: 70
Batch: 71
Batch: 72
Batch: 73
Batch: 74
Batch: 75
Batch: 76
Batch: 77
Batch: 78
Batch: 79
Batch: 80
Batch: 81
Batch: 82
Batch: 83
Batch: 84
Batch: 85
Batch: 86
Batch: 87
Batch: 88
Batch: 89
Batch: 90
Batch: 91
Batch: 92
Batch: 93
Batch: 94
Batch: 95
Batch: 96
Batch: 97
Batch: 98
Batch: 99
Batch: 100

In [ ]:
concated_embeds_CLSs = torch.cat(embeds_CLSs, 0)
concated_embeds_mean = torch.cat(embeds_mean, 0)

In [ ]:
embedding_CLS = concated_embeds_CLSs.tolist()
embedding_mean = concated_embeds_mean.tolist()

In [ ]:
df['embeddingCLS'] = embedding_CLS
df['embeddingMean'] = embedding_mean

# Guardamos el df

In [ ]:
df.to_csv('/content/drive/MyDrive/peru_larco/Data/base_clean/base_global_bert.csv', sep=';', index=False)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/peru_larco/Data/base_clean/base_global_bert.csv', sep=';')

In [ ]:
df

,catalogation_id,culture_cl,description,tokens,cronology_time,embeddingCLS,embeddingMean
0,ML020107,Sican,botella doble cuerpo asa puente cintada silbad...,[CLS] botella doble cuerpo asa puente cinta ##...,7,"[-0.2504190504550934, -1.0645008087158203, 0.4...","[-0.07850763201713562, -0.7908726930618286, 0...."
1,ML020108,Sican,botella doble pico asa puente cintada escultor...,[CLS] botella doble pico asa puente cinta ##da...,7,"[0.12584586441516876, -0.9540853500366211, 0.2...","[0.31227222084999084, -0.46838024258613586, -0..."
2,ML020109,Sican,botella gollete asa puente cintada protoma sil...,[CLS] botella gol ##let ##e asa puente cinta #...,7,"[0.8280690312385559, -0.22550193965435028, 1.0...","[0.650943398475647, -0.4512614607810974, -0.22..."
3,ML020110,Sican,botella gollete asa puente cintada protoma sil...,[CLS] botella gol ##let ##e asa puente cinta #...,7,"[0.9305453896522522, -1.2327792644500732, -0.0...","[0.2903463840484619, -0.46815067529678345, -0...."
4,ML020111,Sican,botella gollete asa puente cintada protoma sil...,[CLS] botella gol ##let ##e asa puente cinta #...,7,"[0.2350309044122696, -0.06415646523237228, 0.4...","[0.09728316962718964, -0.11855237185955048, 0...."
...,...,...,...,...,...,...,...
32623,ML035498,Tiahuanaco,botella gollete asa lateral con disenos geomet...,[CLS] botella gol ##let ##e asa lateral con di...,6,"[1.0124317407608032, -0.6778260469436646, 0.78...","[0.6625721454620361, -0.43327921628952026, 0.1..."
32624,ML038832,Tiahuanaco,plato con disenos geometricos de lineas horizo...,[CLS] plato con dise ##nos geo ##met ##rico ##...,6,"[0.3006052076816559, -1.2429275512695312, 0.89...","[0.01702764816582203, -0.9985824823379517, 0.9..."
32625,ML038833,Tiahuanaco,plato con disenos geometricos de eses ( s) y l...,[CLS] plato con dise ##nos geo ##met ##rico ##...,6,"[0.6021231412887573, -0.3880494236946106, -0.0...","[0.38921570777893066, -0.20432981848716736, -0..."
32626,ML015241,Cajamarca,cuenco con representar de cabeza estilizada co...,[CLS] cuen ##co con representar de cabeza esti...,5,"[0.5401057600975037, -1.4557368755340576, 0.44...","[0.4593050479888916, -0.9496500492095947, 0.68..."


# Repetimos para la base n150

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/peru_larco/Data/base_n150/base_n150.csv',  sep=';')
df

,Unnamed: 0,catalogation_id,cronology,cronology_time,culture_cl,morfofunctional_category,description,principal_scene,decoration_tecnique_external_body_section1,color_external_body_section1,...,trait_n101,trait_n102,trait_n103,trait_n104,trait_n105,file_path,image_path,description_length,tokens,numero_palabras
0,0,ML020107,Horizonte Medio,7,Sican,botella doble cuerpo asa puente cintada silbadora,botella doble cuerpo asa puente cintada silbad...,NaN,pintado escultorico,crema y naranja,...,1,1,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020107a.jpg,191,botella doble cuerpo asa puente cintada silbad...,21
1,1,ML020108,Horizonte Medio,7,Sican,botella doble pico asa puente cintada escultorica,botella doble pico asa puente cintada escultor...,NaN,pintado escultorico,rojo y naranja,...,1,1,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020108a.jpg,245,botella doble pico asa puente cintada escultor...,29
2,2,ML020109,Horizonte Medio,7,Sican,botella gollete asa puente cintada protoma sil...,botella gollete asa puente cintada protoma sil...,NaN,pintado escultorico,crema y naranja,...,1,1,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020109a.jpg,134,botella gollete asa puente cintada protoma sil...,16
3,3,ML020110,Horizonte Medio,7,Sican,botella gollete asa puente cintada protoma sil...,botella gollete asa puente cintada protoma sil...,NaN,pintado escultorico,crema y naranja,...,1,1,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020110a.jpg,125,botella gollete asa puente cintada protoma sil...,15
4,4,ML020111,Horizonte Medio,7,Sican,botella gollete asa puente cintada protoma sil...,botella gollete asa puente cintada protoma sil...,NaN,pintado escultorico,crema y marron,...,1,1,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020111a.jpg,158,botella gollete asa puente cintada protoma sil...,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32303,33495,ML038664,Horizonte Medio,7,Tiahuanaco,plato,plato con disenos geometricos de escalonados y...,NaN,pintado,naranja,...,1,1,0,1,0,data/tiahuanaco_7/16 ADMINISTRADOR COLECCIONES...,data/images/ML038664a.jpg,58,plato disenos geometricos escalonados triangulos,5
32304,33496,ML038665,Horizonte Medio,7,Tiahuanaco,plato,"plato con disenos geometricos de semicirculo, ...",NaN,pintado,naranja,...,1,1,0,1,0,data/tiahuanaco_7/16 ADMINISTRADOR COLECCIONES...,data/images/ML038665a.jpg,67,plato disenos geometricos semicirculo escalona...,6
32305,33497,ML038666,Horizonte Medio,7,Tiahuanaco,plato,plato con representar de cabeza de personajes ...,NaN,pintado,naranja,...,1,1,0,1,0,data/tiahuanaco_7/16 ADMINISTRADOR COLECCIONES...,data/images/ML038666a.jpg,56,plato representar cabeza personajes zoomorfos,5
32306,33498,ML038667,Horizonte Medio,7,Tiahuanaco,plato,plato con disenos geometricos de semicirculo.,NaN,pintado,naranja,...,1,1,0,1,0,data/tiahuanaco_7/16 ADMINISTRADOR COLECCIONES...,data/images/ML038667a.jpg,45,plato disenos geometricos semicirculo,4


In [ ]:
df = df[~df.description.isnull()]
df['description_length'] = df.description.apply(lambda x: len(x))
max(df.description_length)

1342

In [ ]:
descriptions = df.tokens.tolist()

In [ ]:
import numpy as np
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
model_name = 'dccuchile/bert-base-spanish-wwm-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of BertModel were not initialized from the model checkpoint at dccuchile/ber

In [ ]:
def prepare_data_for_bert(texts, tokenizer, max_length):
    input_ids = []
    attention_masks = []
    encodeds_data = []

    for text in texts:
        encoded_data = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_length,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        input_ids.append(encoded_data['input_ids'])
        attention_masks.append(encoded_data['attention_mask'])
        encodeds_data.append(encoded_data)

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)

    return input_ids, attention_masks, encodeds_data

In [ ]:
input_ids, attention_masks, encodeds_data = prepare_data_for_bert(descriptions, tokenizer, 256)

In [ ]:
df

,Unnamed: 0,catalogation_id,cronology,cronology_time,culture_cl,morfofunctional_category,description,principal_scene,decoration_tecnique_external_body_section1,color_external_body_section1,...,trait_n101,trait_n102,trait_n103,trait_n104,trait_n105,file_path,image_path,description_length,tokens,numero_palabras
0,0,ML020107,Horizonte Medio,7,Sican,botella doble cuerpo asa puente cintada silbadora,botella doble cuerpo asa puente cintada silbad...,NaN,pintado escultorico,crema y naranja,...,1,1,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020107a.jpg,191,botella doble cuerpo asa puente cintada silbad...,21
1,1,ML020108,Horizonte Medio,7,Sican,botella doble pico asa puente cintada escultorica,botella doble pico asa puente cintada escultor...,NaN,pintado escultorico,rojo y naranja,...,1,1,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020108a.jpg,245,botella doble pico asa puente cintada escultor...,29
2,2,ML020109,Horizonte Medio,7,Sican,botella gollete asa puente cintada protoma sil...,botella gollete asa puente cintada protoma sil...,NaN,pintado escultorico,crema y naranja,...,1,1,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020109a.jpg,134,botella gollete asa puente cintada protoma sil...,16
3,3,ML020110,Horizonte Medio,7,Sican,botella gollete asa puente cintada protoma sil...,botella gollete asa puente cintada protoma sil...,NaN,pintado escultorico,crema y naranja,...,1,1,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020110a.jpg,125,botella gollete asa puente cintada protoma sil...,15
4,4,ML020111,Horizonte Medio,7,Sican,botella gollete asa puente cintada protoma sil...,botella gollete asa puente cintada protoma sil...,NaN,pintado escultorico,crema y marron,...,1,1,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020111a.jpg,158,botella gollete asa puente cintada protoma sil...,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32303,33495,ML038664,Horizonte Medio,7,Tiahuanaco,plato,plato con disenos geometricos de escalonados y...,NaN,pintado,naranja,...,1,1,0,1,0,data/tiahuanaco_7/16 ADMINISTRADOR COLECCIONES...,data/images/ML038664a.jpg,58,plato disenos geometricos escalonados triangulos,5
32304,33496,ML038665,Horizonte Medio,7,Tiahuanaco,plato,"plato con disenos geometricos de semicirculo, ...",NaN,pintado,naranja,...,1,1,0,1,0,data/tiahuanaco_7/16 ADMINISTRADOR COLECCIONES...,data/images/ML038665a.jpg,67,plato disenos geometricos semicirculo escalona...,6
32305,33497,ML038666,Horizonte Medio,7,Tiahuanaco,plato,plato con representar de cabeza de personajes ...,NaN,pintado,naranja,...,1,1,0,1,0,data/tiahuanaco_7/16 ADMINISTRADOR COLECCIONES...,data/images/ML038666a.jpg,56,plato representar cabeza personajes zoomorfos,5
32306,33498,ML038667,Horizonte Medio,7,Tiahuanaco,plato,plato con disenos geometricos de semicirculo.,NaN,pintado,naranja,...,1,1,0,1,0,data/tiahuanaco_7/16 ADMINISTRADOR COLECCIONES...,data/images/ML038667a.jpg,45,plato disenos geometricos semicirculo,4


In [ ]:
tokenizer.convert_ids_to_tokens(encodeds_data[0]['input_ids'][0])

['[CLS]',
 'botella',
 'doble',
 'cuerpo',
 'asa',
 'puente',
 'cinta',
 '##da',
 'sil',
 '##bado',
 '##ra',
 'representar',
 'personaje',
 'tocado',
 'pintura',
 'facial',
 'tapar',
 '##ra',
 '##bo',
 'coge',
 'hombros',
 'personaje',
 'casco',
 'pintura',
 'facial',
 'collar',
 'tun',
 '##ica',
 '[SEP]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '

In [ ]:
attention_masks

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])

In [ ]:
df

,Unnamed: 0,catalogation_id,cronology,cronology_time,culture_cl,morfofunctional_category,description,principal_scene,decoration_tecnique_external_body_section1,color_external_body_section1,...,trait_n101,trait_n102,trait_n103,trait_n104,trait_n105,file_path,image_path,description_length,tokens,numero_palabras
0,0,ML020107,Horizonte Medio,7,Sican,botella doble cuerpo asa puente cintada silbadora,botella doble cuerpo asa puente cintada silbad...,NaN,pintado escultorico,crema y naranja,...,1,1,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020107a.jpg,191,botella doble cuerpo asa puente cintada silbad...,21
1,1,ML020108,Horizonte Medio,7,Sican,botella doble pico asa puente cintada escultorica,botella doble pico asa puente cintada escultor...,NaN,pintado escultorico,rojo y naranja,...,1,1,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020108a.jpg,245,botella doble pico asa puente cintada escultor...,29
2,2,ML020109,Horizonte Medio,7,Sican,botella gollete asa puente cintada protoma sil...,botella gollete asa puente cintada protoma sil...,NaN,pintado escultorico,crema y naranja,...,1,1,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020109a.jpg,134,botella gollete asa puente cintada protoma sil...,16
3,3,ML020110,Horizonte Medio,7,Sican,botella gollete asa puente cintada protoma sil...,botella gollete asa puente cintada protoma sil...,NaN,pintado escultorico,crema y naranja,...,1,1,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020110a.jpg,125,botella gollete asa puente cintada protoma sil...,15
4,4,ML020111,Horizonte Medio,7,Sican,botella gollete asa puente cintada protoma sil...,botella gollete asa puente cintada protoma sil...,NaN,pintado escultorico,crema y marron,...,1,1,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020111a.jpg,158,botella gollete asa puente cintada protoma sil...,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32303,33495,ML038664,Horizonte Medio,7,Tiahuanaco,plato,plato con disenos geometricos de escalonados y...,NaN,pintado,naranja,...,1,1,0,1,0,data/tiahuanaco_7/16 ADMINISTRADOR COLECCIONES...,data/images/ML038664a.jpg,58,plato disenos geometricos escalonados triangulos,5
32304,33496,ML038665,Horizonte Medio,7,Tiahuanaco,plato,"plato con disenos geometricos de semicirculo, ...",NaN,pintado,naranja,...,1,1,0,1,0,data/tiahuanaco_7/16 ADMINISTRADOR COLECCIONES...,data/images/ML038665a.jpg,67,plato disenos geometricos semicirculo escalona...,6
32305,33497,ML038666,Horizonte Medio,7,Tiahuanaco,plato,plato con representar de cabeza de personajes ...,NaN,pintado,naranja,...,1,1,0,1,0,data/tiahuanaco_7/16 ADMINISTRADOR COLECCIONES...,data/images/ML038666a.jpg,56,plato representar cabeza personajes zoomorfos,5
32306,33498,ML038667,Horizonte Medio,7,Tiahuanaco,plato,plato con disenos geometricos de semicirculo.,NaN,pintado,naranja,...,1,1,0,1,0,data/tiahuanaco_7/16 ADMINISTRADOR COLECCIONES...,data/images/ML038667a.jpg,45,plato disenos geometricos semicirculo,4


In [ ]:
df['tokens'] = [' '.join(tokenizer.convert_ids_to_tokens(encodeds_data[i]['input_ids'][0])) for i in range(len(encodeds_data))]

In [ ]:
df

,Unnamed: 0,catalogation_id,cronology,cronology_time,culture_cl,morfofunctional_category,description,principal_scene,decoration_tecnique_external_body_section1,color_external_body_section1,...,trait_n101,trait_n102,trait_n103,trait_n104,trait_n105,file_path,image_path,description_length,tokens,numero_palabras
0,0,ML020107,Horizonte Medio,7,Sican,botella doble cuerpo asa puente cintada silbadora,botella doble cuerpo asa puente cintada silbad...,NaN,pintado escultorico,crema y naranja,...,1,1,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020107a.jpg,191,[CLS] botella doble cuerpo asa puente cinta ##...,21
1,1,ML020108,Horizonte Medio,7,Sican,botella doble pico asa puente cintada escultorica,botella doble pico asa puente cintada escultor...,NaN,pintado escultorico,rojo y naranja,...,1,1,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020108a.jpg,245,[CLS] botella doble pico asa puente cinta ##da...,29
2,2,ML020109,Horizonte Medio,7,Sican,botella gollete asa puente cintada protoma sil...,botella gollete asa puente cintada protoma sil...,NaN,pintado escultorico,crema y naranja,...,1,1,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020109a.jpg,134,[CLS] botella gol ##let ##e asa puente cinta #...,16
3,3,ML020110,Horizonte Medio,7,Sican,botella gollete asa puente cintada protoma sil...,botella gollete asa puente cintada protoma sil...,NaN,pintado escultorico,crema y naranja,...,1,1,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020110a.jpg,125,[CLS] botella gol ##let ##e asa puente cinta #...,15
4,4,ML020111,Horizonte Medio,7,Sican,botella gollete asa puente cintada protoma sil...,botella gollete asa puente cintada protoma sil...,NaN,pintado escultorico,crema y marron,...,1,1,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020111a.jpg,158,[CLS] botella gol ##let ##e asa puente cinta #...,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32303,33495,ML038664,Horizonte Medio,7,Tiahuanaco,plato,plato con disenos geometricos de escalonados y...,NaN,pintado,naranja,...,1,1,0,1,0,data/tiahuanaco_7/16 ADMINISTRADOR COLECCIONES...,data/images/ML038664a.jpg,58,[CLS] plato dise ##nos geo ##met ##rico ##s es...,5
32304,33496,ML038665,Horizonte Medio,7,Tiahuanaco,plato,"plato con disenos geometricos de semicirculo, ...",NaN,pintado,naranja,...,1,1,0,1,0,data/tiahuanaco_7/16 ADMINISTRADOR COLECCIONES...,data/images/ML038665a.jpg,67,[CLS] plato dise ##nos geo ##met ##rico ##s se...,6
32305,33497,ML038666,Horizonte Medio,7,Tiahuanaco,plato,plato con representar de cabeza de personajes ...,NaN,pintado,naranja,...,1,1,0,1,0,data/tiahuanaco_7/16 ADMINISTRADOR COLECCIONES...,data/images/ML038666a.jpg,56,[CLS] plato representar cabeza personajes zoo ...,5
32306,33498,ML038667,Horizonte Medio,7,Tiahuanaco,plato,plato con disenos geometricos de semicirculo.,NaN,pintado,naranja,...,1,1,0,1,0,data/tiahuanaco_7/16 ADMINISTRADOR COLECCIONES...,data/images/ML038667a.jpg,45,[CLS] plato dise ##nos geo ##met ##rico ##s se...,4


In [ ]:
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
dataset = TensorDataset(input_ids, attention_masks)
data_loader = DataLoader(dataset, batch_size=8, shuffle=True)

In [ ]:
torch.cuda.is_available()

True

In [ ]:
torch.cuda.empty_cache()

In [ ]:
torch.cuda.memory_allocated()

0

In [ ]:
device = torch.device('cuda')

In [ ]:
model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(31002, 768, padding_idx=1)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [ ]:
def get_embeddings(model, data_loader):
    model.eval()
    embeds_CLSs = []
    embeds_mean = []

    for i, batch in enumerate(data_loader):
        print(f'Batch: {i}')

        b_input_ids = batch[0].to(device)
        b_attention_masks = batch[1].to(device)

        with torch.no_grad():
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_attention_masks)[0]
            CLSs = outputs[:, 0, :]
            means = outputs.mean(dim=1)

            # Guardamos los embeddings
            embeds_CLSs.append(CLSs)
            embeds_mean.append(means)

In [ ]:
model.eval()
embeds_CLSs = []
embeds_mean = []
embeds_max_over_time = []

for i, batch in enumerate(data_loader):
    print(f'Batch: {i}')

    b_input_ids = batch[0].to(device)
    b_attention_masks = batch[1].to(device)

    with torch.no_grad():
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_attention_masks)[0]
        CLSs = outputs[:, 0, :]
        means = outputs.mean(dim=1)
        max_over_time = outputs.max(dim=1)

        # Guardamos los embeddings
        embeds_CLSs.append(CLSs)
        embeds_mean.append(means)
        embeds_max_over_time.append(max_over_time)

Batch: 0
Batch: 1
Batch: 2
Batch: 3
Batch: 4
Batch: 5
Batch: 6
Batch: 7
Batch: 8
Batch: 9
Batch: 10
Batch: 11
Batch: 12
Batch: 13
Batch: 14
Batch: 15
Batch: 16
Batch: 17
Batch: 18
Batch: 19
Batch: 20
Batch: 21
Batch: 22
Batch: 23
Batch: 24
Batch: 25
Batch: 26
Batch: 27
Batch: 28
Batch: 29
Batch: 30
Batch: 31
Batch: 32
Batch: 33
Batch: 34
Batch: 35
Batch: 36
Batch: 37
Batch: 38
Batch: 39
Batch: 40
Batch: 41
Batch: 42
Batch: 43
Batch: 44
Batch: 45
Batch: 46
Batch: 47
Batch: 48
Batch: 49
Batch: 50
Batch: 51
Batch: 52
Batch: 53
Batch: 54
Batch: 55
Batch: 56
Batch: 57
Batch: 58
Batch: 59
Batch: 60
Batch: 61
Batch: 62
Batch: 63
Batch: 64
Batch: 65
Batch: 66
Batch: 67
Batch: 68
Batch: 69
Batch: 70
Batch: 71
Batch: 72
Batch: 73
Batch: 74
Batch: 75
Batch: 76
Batch: 77
Batch: 78
Batch: 79
Batch: 80
Batch: 81
Batch: 82
Batch: 83
Batch: 84
Batch: 85
Batch: 86
Batch: 87
Batch: 88
Batch: 89
Batch: 90
Batch: 91
Batch: 92
Batch: 93
Batch: 94
Batch: 95
Batch: 96
Batch: 97
Batch: 98
Batch: 99
Batch: 100

In [ ]:
concated_embeds_CLSs = torch.cat(embeds_CLSs, 0)
concated_embeds_mean = torch.cat(embeds_mean, 0)

In [ ]:
embedding_CLS = concated_embeds_CLSs.tolist()
embedding_mean = concated_embeds_mean.tolist()

In [ ]:
df['embeddingCLS'] = embedding_CLS
df['embeddingMean'] = embedding_mean

In [ ]:
df.to_csv('/content/drive/MyDrive/peru_larco/Data/base_n150/base_n150_bert.csv', sep=';', index=False)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/peru_larco/Data/base_n150/base_n150_bert.csv', sep=';')

In [ ]:
df

,Unnamed: 0,catalogation_id,cronology,cronology_time,culture_cl,morfofunctional_category,description,principal_scene,decoration_tecnique_external_body_section1,color_external_body_section1,...,trait_n103,trait_n104,trait_n105,file_path,image_path,description_length,tokens,numero_palabras,embeddingCLS,embeddingMean
0,0,ML020107,Horizonte Medio,7,Sican,botella doble cuerpo asa puente cintada silbadora,botella doble cuerpo asa puente cintada silbad...,NaN,pintado escultorico,crema y naranja,...,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020107a.jpg,191,[CLS] botella doble cuerpo asa puente cinta ##...,21,"[0.07349269092082977, -0.7621917128562927, 0.0...","[0.0001518863718956709, -0.38347482681274414, ..."
1,1,ML020108,Horizonte Medio,7,Sican,botella doble pico asa puente cintada escultorica,botella doble pico asa puente cintada escultor...,NaN,pintado escultorico,rojo y naranja,...,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020108a.jpg,245,[CLS] botella doble pico asa puente cinta ##da...,29,"[-0.16221104562282562, -0.5011821985244751, 0....","[-0.012210816144943237, -0.4184707999229431, -..."
2,2,ML020109,Horizonte Medio,7,Sican,botella gollete asa puente cintada protoma sil...,botella gollete asa puente cintada protoma sil...,NaN,pintado escultorico,crema y naranja,...,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020109a.jpg,134,[CLS] botella gol ##let ##e asa puente cinta #...,16,"[-0.3746039569377899, -0.3084278702735901, -0....","[-0.24044576287269592, 0.03168115019798279, -0..."
3,3,ML020110,Horizonte Medio,7,Sican,botella gollete asa puente cintada protoma sil...,botella gollete asa puente cintada protoma sil...,NaN,pintado escultorico,crema y naranja,...,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020110a.jpg,125,[CLS] botella gol ##let ##e asa puente cinta #...,15,"[0.22687536478042603, -0.7911874651908875, -0....","[0.03867929428815842, -1.0952448844909668, -0...."
4,4,ML020111,Horizonte Medio,7,Sican,botella gollete asa puente cintada protoma sil...,botella gollete asa puente cintada protoma sil...,NaN,pintado escultorico,crema y marron,...,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020111a.jpg,158,[CLS] botella gol ##let ##e asa puente cinta #...,18,"[-0.1266753375530243, -0.5457064509391785, 0.3...","[0.10204872488975525, -0.17494848370552063, -0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32303,33495,ML038664,Horizonte Medio,7,Tiahuanaco,plato,plato con disenos geometricos de escalonados y...,NaN,pintado,naranja,...,0,1,0,data/tiahuanaco_7/16 ADMINISTRADOR COLECCIONES...,data/images/ML038664a.jpg,58,[CLS] plato dise ##nos geo ##met ##rico ##s es...,5,"[-0.3734225332736969, -0.6006585955619812, 0.1...","[-0.3264613151550293, -0.3642657697200775, 0.0..."
32304,33496,ML038665,Horizonte Medio,7,Tiahuanaco,plato,"plato con disenos geometricos de semicirculo, ...",NaN,pintado,naranja,...,0,1,0,data/tiahuanaco_7/16 ADMINISTRADOR COLECCIONES...,data/images/ML038665a.jpg,67,[CLS] plato dise ##nos geo ##met ##rico ##s se...,6,"[0.17742006480693817, -0.8158581852912903, -0....","[0.1764347404241562, -0.23271816968917847, -0...."
32305,33497,ML038666,Horizonte Medio,7,Tiahuanaco,plato,plato con representar de cabeza de personajes ...,NaN,pintado,naranja,...,0,1,0,data/tiahuanaco_7/16 ADMINISTRADOR COLECCIONES...,data/images/ML038666a.jpg,56,[CLS] plato representar cabeza personajes zoo ...,5,"[0.1151195764541626, -0.7184727787971497, 0.19...","[0.5330296754837036, -0.14330625534057617, -0...."
32306,33498,ML038667,Horizonte Medio,7,Tiahuanaco,plato,plato con disenos geometricos de semicirculo.,NaN,pintado,naranja,...,0,1,0,data/tiahuanaco_7/16 ADMINISTRADOR COLECCIONES...,data/images/ML038667a.jpg,45,[CLS] plato dise ##nos geo ##met ##rico ##s se...,4,"[-0.9056000113487244, -0.90920090675354, 0.197...","[-0.5616457462310791, -0.32798314094543457, -0.